# Import libraries and model

In [49]:
from os import listdir
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import gc
# import gensim.downloader as api

import torch
import torch.nn as nn

from math import ceil
import torch.optim as optim

nltk.download('stopwords')
nltk.download('wordnet')

# Load GloVe model with Gensim's API - Twitter specific embedding
# embeddings_model = api.load("glove-twitter-200")  # 200-dimensional GloVe embeddings

#To check that T4 GPU is connected
#!nvidia-smi

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\panda\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\panda\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [50]:
# %pip install "modin[all]"
# import modin.pandas as pd

In [51]:
import pandas as pd

In [52]:
# %pip install swifter
# import swifter

In [53]:
# import os
# os.environ["MODIN_ENGINE"] = "dask"  # Options: "ray" or "dask"
# import modin.pandas as pd

In [54]:
# import dask.dataframe as pd

In [55]:
import swifter

# Data preprocessing

In [56]:
# Read all training files and concatenate them into one dataframe

#import os
#print(os.getcwd())

li = []
i = 0
for filename in listdir("train_tweets"):
    if filename != '.ipynb_checkpoints':
        print(filename)
        df = pd.read_csv("train_tweets/" + filename)
        # df.drop(columns=['Timestamp'], inplace=True)
        # drop unused column(s)
        df['MatchID'] = str(i)
        df['ID'] = str(i)+ '_' + df['PeriodID'].astype(str)
        # makes sure that the match IDs are ordered from 0,1,2... with no missing values
        i+=1
        li.append(df)
df = pd.concat(li, ignore_index=True)
#print(len(df))
df

ArgentinaBelgium72.csv
ArgentinaGermanyFinal77.csv
AustraliaNetherlands29.csv
AustraliaSpain34.csv
BelgiumSouthKorea59.csv
CameroonBrazil36.csv
FranceGermany70.csv
FranceNigeria66.csv
GermanyAlgeria67.csv
GermanyBrazil74.csv
GermanyUSA57.csv
HondurasSwitzerland54.csv
MexicoCroatia37.csv
NetherlandsChile35.csv
PortugalGhana58.csv
USASlovenia2010.csv


,ID,MatchID,PeriodID,EventType,Timestamp,Tweet
0,0_0,0,0,0,1404575400000,RT @2014WorIdCup: Argentina vs Belgium\n\nWho ...
1,0_0,0,0,0,1404575400000,@elijahman_ time to focus on Belgium winning t...
2,0_0,0,0,0,1404575400000,RT @FIFAWorldCup: GLOBAL STADIUM: #Joinin with...
3,0_0,0,0,0,1404575400000,RT @CatholicNewsSvc: #PopeFrancis. Uh-oh. Arge...
4,0_0,0,0,0,1404575400000,RT @soccerdotcom: If he scores vs #BEL we'll a...
...,...,...,...,...,...,...
5056045,15_129,15,129,0,1276876799000,RT @nytimes FIFA World Cup -- Final Score: U.S...
5056046,15_129,15,129,0,1276876799000,Ugh!!! should've been 3-2 USA! #worldcup
5056047,15_129,15,129,0,1276876799000,RT @jaclynkeough: Ha! RT @someecards I'd rathe...
5056048,15_129,15,129,0,1276876799000,RT @gustavaulia: So many surprises at worldcup...


In [57]:
# swifter.register_parallel_series_accessor(df['Tweet'])

In [58]:
del li

In [59]:
gc.collect()

2548

In [60]:
# Preprocessing of tweet
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^a-z\s]', '', text)
    # Remove numbers
    # text = re.sub(r'\d+', '', text)
    # Tokenization
    words = text.split()
    # Remove stopwords
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    # Lemmatization
    # words = [lemmatizer.lemmatize(word) for word in words]
    return ''.join(words)

In [61]:
# Apply preprocessing to each tweet
df['Tweet'] = df['Tweet'].swifter.apply(preprocess_text)
df

Pandas Apply:   0%|          | 0/5056050 [00:00<?, ?it/s]

,ID,MatchID,PeriodID,EventType,Timestamp,Tweet
0,0_0,0,0,0,1404575400000,rtworidcupargentinavbelgiumwinhttptcoleuwtbzyy
1,0_0,0,0,0,1404575400000,elijahmantimefocusbelgiumwinningworldcup
2,0_0,0,0,0,1404575400000,rtfifaworldcupglobalstadiumjoininworldcupcover...
3,0_0,0,0,0,1404575400000,rtcatholicnewssvcpopefrancisuhohargentinavbelg...
4,0_0,0,0,0,1404575400000,rtsoccerdotcomscorevbelwellawardmessisignedbra...
...,...,...,...,...,...,...
5056045,15_129,15,129,0,1276876799000,rtnytimesfifaworldcupfinalscoreusloveniatiegam...
5056046,15_129,15,129,0,1276876799000,ughshouldveusaworldcup
5056047,15_129,15,129,0,1276876799000,rtjaclynkeoughhartsomeecardsidratherdiefirings...
5056048,15_129,15,129,0,1276876799000,rtgustavauliamanysurpriseworldcuptimewowwbykyg...


In [62]:
key_i = [0]
keys = {}
tot_counts = {}
def get_ngram_counts(string: str, n=5,top=200, key_i=None):
    counts = {}
    for i in range(len(string)-n):
        k = string[i:i+n]
        if k not in keys:
            keys[k] = key_i[0]
            key_i[0] += 1
        key = keys[k]
        tot_counts[key] = tot_counts.get(key, 0) + 1
        counts[key] = counts.get(key, 0) + 1
    return counts

In [63]:
df['counts'] = df['Tweet'].apply(get_ngram_counts, key_i=key_i, n=8)

In [64]:
df = df.drop(columns=['Tweet'])

In [65]:
df = df.drop(columns=['Timestamp'])

In [66]:
i_keys = {v: k for k, v in keys.items()}

In [67]:
top_200 = sorted(tot_counts.keys(), key=tot_counts.__getitem__, reverse=True)[:200]

In [68]:
def filter_count(counts):    
    return {k: v for k, v in counts.items() if k in top_200}

In [69]:
df['counts'] = df['counts'].swifter.apply(filter_count)

Pandas Apply:   0%|          | 0/5056050 [00:00<?, ?it/s]

In [70]:
def to_ndarr(counts):
    res = np.zeros((200,))
    for k, v in counts.items():
        res[top_200.index(k)] = v
    return res

In [71]:
df['counts'] = df['counts'].swifter.apply(to_ndarr)

Pandas Apply:   0%|          | 0/5056050 [00:00<?, ?it/s]

In [72]:
df.head()

,ID,MatchID,PeriodID,EventType,counts
0,0_0,0,0,0,"[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,0_0,0,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,0_0,0,0,0,"[2.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,0_0,0,0,0,"[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,0_0,0,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


# Tweet Embeddings

In [73]:
# by now should have df with columns: ID, match id, period id, Event Type, tweet_vector. Tweet_vector is just 200 columns
df

,ID,MatchID,PeriodID,EventType,counts
0,0_0,0,0,0,"[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,0_0,0,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,0_0,0,0,0,"[2.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,0_0,0,0,0,"[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,0_0,0,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...
5056045,15_129,15,129,0,"[2.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
5056046,15_129,15,129,0,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5056047,15_129,15,129,0,"[1.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
5056048,15_129,15,129,0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [74]:
# group the tweets into their corresponding periods to generate an average embedding vector for each period
# so there are no duplicate period id rows per match
# decreases size of data + makes it easier to fit into LSTM model
df = df.groupby(['MatchID', 'PeriodID', 'ID']).sum().reset_index()
df.drop(columns=['ID'], inplace=True) 
df['MatchID'] = df['MatchID'].astype(int)
df['PeriodID'] = df['PeriodID'].astype(int)
# need to convert to int before sorting
df.sort_values(by=['MatchID', 'PeriodID'], inplace=True)
df.reset_index(drop=True, inplace=True)


In [75]:
df.loc[df["EventType"] != 0, "EventType"] = 1
df

,MatchID,PeriodID,EventType,counts
0,0,0,0,"[646.0, 771.0, 736.0, 133.0, 7.0, 74.0, 29.0, ..."
1,0,1,0,"[496.0, 663.0, 602.0, 102.0, 1.0, 61.0, 29.0, ..."
2,0,2,0,"[415.0, 626.0, 549.0, 97.0, 3.0, 47.0, 22.0, 1..."
3,0,3,0,"[645.0, 798.0, 702.0, 144.0, 5.0, 51.0, 30.0, ..."
4,0,4,0,"[719.0, 929.0, 822.0, 190.0, 7.0, 80.0, 28.0, ..."
...,...,...,...,...
2132,15,125,1,"[637.0, 0.0, 0.0, 348.0, 0.0, 16.0, 0.0, 44.0,..."
2133,15,126,1,"[525.0, 0.0, 0.0, 299.0, 0.0, 14.0, 0.0, 36.0,..."
2134,15,127,1,"[557.0, 2.0, 2.0, 292.0, 2.0, 17.0, 0.0, 57.0,..."
2135,15,128,1,"[491.0, 2.0, 2.0, 234.0, 0.0, 18.0, 0.0, 48.0,..."


In [78]:
largest_count = df.counts.swifter.apply(np.max).max()
largest_count

Pandas Apply:   0%|          | 0/2137 [00:00<?, ?it/s]

4202.0

In [80]:
df['tweet_vector'] = df.counts.swifter.apply(lambda v: v / largest_count)

Pandas Apply:   0%|          | 0/2137 [00:00<?, ?it/s]

In [82]:
df.drop(columns=['counts'], inplace=True)

In [83]:
df

,MatchID,PeriodID,EventType,tweet_vector
0,0,0,0,"[0.15373631603998097, 0.18348405521180391, 0.1..."
1,0,1,0,"[0.11803902903379343, 0.1577820085673489, 0.14..."
2,0,2,0,"[0.09876249405045216, 0.1489766777724893, 0.13..."
3,0,3,0,"[0.15349833412660638, 0.18990956687291766, 0.1..."
4,0,4,0,"[0.17110899571632557, 0.2210851975249881, 0.19..."
...,...,...,...,...
2132,15,125,1,"[0.15159447881960972, 0.0, 0.0, 0.082817705854..."
2133,15,126,1,"[0.12494050452165635, 0.0, 0.0, 0.071156592099..."
2134,15,127,1,"[0.13255592574964303, 0.00047596382674916705, ..."
2135,15,128,1,"[0.11684911946692052, 0.00047596382674916705, ..."


# Separate Train and Test data

In [85]:
# train on of the first 13 of 16 matches (16*0.8=12.8~=13)
# and the test data would be the last 3 matches. 
# Before submitting on Kaggle we should train on full dataset, so al 16 matches
train_percentage = 0.8
unique_match_ids = df['MatchID'].unique()
print(unique_match_ids)
num_matches_training = int(ceil(len(unique_match_ids)*train_percentage))
print(num_matches_training)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
13


In [86]:
target_match_id = num_matches_training
# target_match_id is first match id that will appear in test set
# all matches from target_match_id and after will be in test test
print(target_match_id)

13


In [88]:

#df2 = df['MatchID'] == 15
#df2

In [87]:
# row_index is first row with match id target_match_id
# row_index is then the first row of the matches that will go to the test


row_index = (df['MatchID'] == target_match_id).idxmax()
#row_index = df[df['MatchID'] == target_match_id].first_valid_index()
df_X_train = df[:row_index].copy()
df_X_test = df[row_index:].copy()


In [88]:
df_y_train = df_X_train['EventType']
df_y_test = df_X_test['EventType']

In [89]:
df_y_train

0       0
1       0
2       0
3       0
4       0
       ..
1742    1
1743    1
1744    1
1745    1
1746    1
Name: EventType, Length: 1747, dtype: int64

In [90]:
df_y_test.reset_index(drop=True, inplace=True)
df_y_test

0      0
1      0
2      1
3      1
4      1
      ..
385    1
386    1
387    1
388    1
389    0
Name: EventType, Length: 390, dtype: int64

In [91]:
df_X_train.drop(['EventType'], axis=1, inplace=True)
df_X_test.drop(['EventType'], axis=1, inplace=True)

In [92]:
df_X_train

,MatchID,PeriodID,tweet_vector
0,0,0,"[0.15373631603998097, 0.18348405521180391, 0.1..."
1,0,1,"[0.11803902903379343, 0.1577820085673489, 0.14..."
2,0,2,"[0.09876249405045216, 0.1489766777724893, 0.13..."
3,0,3,"[0.15349833412660638, 0.18990956687291766, 0.1..."
4,0,4,"[0.17110899571632557, 0.2210851975249881, 0.19..."
...,...,...,...
1742,12,125,"[0.47620180866254164, 0.0, 0.0, 0.041170871013..."
1743,12,126,"[0.6573060447405997, 0.0, 0.0, 0.1432651118514..."
1744,12,127,"[0.6587339362208472, 0.0, 0.0, 0.1508805330794..."
1745,12,128,"[0.5735364112327463, 0.0, 0.0, 0.1399333650642..."


In [93]:
df_X_test.reset_index(drop=True, inplace=True)
df_X_test

,MatchID,PeriodID,tweet_vector
0,13,0,"[0.02427415516420752, 0.0, 0.0, 0.002855782960..."
1,13,1,"[0.020942408376963352, 0.0, 0.0, 0.00118990956..."
2,13,2,"[0.02617801047120419, 0.0, 0.0, 0.003569728700..."
3,13,3,"[0.029271775345073775, 0.0, 0.0, 0.00404569252..."
4,13,4,"[0.02641599238457877, 0.0, 0.0, 0.003569728700..."
...,...,...,...
385,15,125,"[0.15159447881960972, 0.0, 0.0, 0.082817705854..."
386,15,126,"[0.12494050452165635, 0.0, 0.0, 0.071156592099..."
387,15,127,"[0.13255592574964303, 0.00047596382674916705, ..."
388,15,128,"[0.11684911946692052, 0.00047596382674916705, ..."


In [94]:
# now df_X_train and df_X_test should have columns MatchID, PeriodID, tweet_vector. Tweet_vector is just 200 columns
# df_y_train and df_y_test should have 1 column, EventType
# the matchids are grouped together so all the rows of the same
# match ids are grouped next to each other, and the periodID are ordered chronologically.

In [95]:
# not working on my machine so I keep DF

# now we have df_X_train, df_X_test, df_y_train, df_y_test
# we no longer need df so we should free up the memory
# del df  # remove reference to the original DataFrame
# gc.collect()  # force garbage collection to free up memory

In [96]:
max_periods = df_X_train.groupby('MatchID')['PeriodID'].max().reset_index()
max_periods
# as we can see not every match has the same number of periods!

,MatchID,PeriodID
0,0,129
1,1,179
2,2,96
3,3,129
4,4,129
5,5,129
6,6,129
7,7,129
8,8,169
9,9,129


In [97]:
max_periods = df_X_test.groupby('MatchID')['PeriodID'].max().reset_index()
max_periods

,MatchID,PeriodID
0,13,129
1,14,129
2,15,129


# Format data for PyTorch LSTM

In [98]:
# input tensor for a PyTorch LSTM should have the shape of (when setting batch_first=True)
# (batch_size, seq_len, num_features) when using the batch_first=True parameter
# batch_size is number of sequences processed at once

# TRY WITHOUT SLIDING WINDOW APPROACH
#    which would mean batch size = number of matches
#    much easier to format for LSTM as 3D tensor
#    dimension of 3D tensor with batch_first=True:(batch_size = num_matches, seq_len = num_periods, num _features = 200)
#    (match_id, period_id, num_features=200)
#     not every match has the same number of periods!, so seq_len can vary between different matches
#     fix: will have to pad with zeroes
# we want tensor[match_id][period_id] to return list len 200 of corresponding tweet vector


In [99]:
df_X_train

,MatchID,PeriodID,tweet_vector
0,0,0,"[0.15373631603998097, 0.18348405521180391, 0.1..."
1,0,1,"[0.11803902903379343, 0.1577820085673489, 0.14..."
2,0,2,"[0.09876249405045216, 0.1489766777724893, 0.13..."
3,0,3,"[0.15349833412660638, 0.18990956687291766, 0.1..."
4,0,4,"[0.17110899571632557, 0.2210851975249881, 0.19..."
...,...,...,...
1742,12,125,"[0.47620180866254164, 0.0, 0.0, 0.041170871013..."
1743,12,126,"[0.6573060447405997, 0.0, 0.0, 0.1432651118514..."
1744,12,127,"[0.6587339362208472, 0.0, 0.0, 0.1508805330794..."
1745,12,128,"[0.5735364112327463, 0.0, 0.0, 0.1399333650642..."


In [100]:
def convert_df_to_3D_tensor(df_X, df_y):
    # df_X should have columns MatchID, PeriodID, tweet_vector. Tweet_vector is just 200 columns
    # rows with same matchID should be grouped together (adjacent rows)
    # df_y should have one column (the EventType)
    # returns tensor_X numpy array already padded! shape: (num_matches, max_num_periods, num _features = 200)
    # and tensor_y of shape: (num_matches, max_num_periods) 
    
    num_matches = len(df_X['MatchID'].unique())
    max_periods = df_X.groupby('MatchID')['PeriodID'].max().reset_index()
    total_max_period = max_periods['PeriodID'].max()
    #total_max_period is max seq len

    tensor_X = np.zeros((num_matches, total_max_period+1, 200))

    tensor_y = np.zeros((num_matches, total_max_period+1))
    print(tensor_X.shape)
    print(tensor_y.shape)
    
    i=0
    previous_match_id = df_X['MatchID'][0]
    for row_index, row in df_X.iterrows():
        match_id = int(row['MatchID'])

        if match_id != previous_match_id:
            i+=1
            previous_match_id = match_id
        
        period_id = int(row['PeriodID'])
        
        features = row.tweet_vector # Skip MatchID and PeriodID
        tensor_X[i, period_id, :] = features
        tensor_y[i,period_id] = df_y[row_index]
        
    return tensor_X, tensor_y

X_train_tensor, y_train_tensor = convert_df_to_3D_tensor(df_X_train, df_y_train)


(13, 180, 200)
(13, 180)


In [101]:
#print(X_train_tensor[0][3])
#print(y_train_tensor[0][3])

In [102]:
# SCALE DATA? minmaxscaler for example!
# SCALING MIGHT BE UNNECESSARY SINCE OUTPUT OF GLOVE TWEET 200 IS ALREADY SCALED BETWEEN -1 AND 1
#scaler = MinMaxScaler()
#tensor = scaler.fit_transform(tensor)

# CONVERT TO PYTORCH TENSOR
X_train_tensor = torch.tensor(X_train_tensor, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_tensor, dtype=torch.float32)

print(X_train_tensor.shape)
print(y_train_tensor.shape)
# X_train_tensor, y_train_tensor are now pytorch tensors

torch.Size([13, 180, 200])
torch.Size([13, 180])


# LSTM Model

In [103]:
# TODO VERIFY ITS CORRECT + MAKE MORE SOPHISTICATED
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout_rate):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_rate)
        self.fc = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out)
        out = out[:,:,:]
        out = self.sigmoid(out) # applying sigmoid to convert to probabilities
        return out.squeeze(-1)

#TODOOOOOOOOOO torch.nn.utils.rnn.pack_padded_sequence. This allows the model to ignore the padded values during computation.

In [104]:
batch_size = 180
batch_idxs = list(range(0,X_train_tensor.shape[0],batch_size))
batched_data = []
for idx in batch_idxs:
    batched_data.append((X_train_tensor[idx:idx+batch_size], y_train_tensor[idx:idx+batch_size]))

# Train model

In [ ]:
hidden_size = 500 # can tune
num_layers = 4 # can tune
dropout_rate = 0.2 # can tune
num_epochs = 500 # can tune
lr = 0.001 # can tune

model = LSTMModel(input_size=200, hidden_size=hidden_size, num_layers=num_layers, dropout_rate=dropout_rate)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCELoss() # great for binary classification
#print(f"Shape of X_train_tensor: {X_train_tensor.shape}")
model.train()
for epoch in range(num_epochs):
    for train, label in batched_data:
        optimizer.zero_grad()
        outputs = model(train)
        #print(f"shape of outputs: {outputs.shape}")
        
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}")

print("Model is trained! (on training data)")

Epoch [0/500], Loss: 0.6934
Epoch [10/500], Loss: 0.6569


# Evaluate on test data

In [150]:
# convert df_X_test and df_y_test to correct format/dimensions
X_test_tensor, y_test_tensor = convert_df_to_3D_tensor(df_X_test, df_y_test)
# CONVERT TO PYTORCH TENSOR
X_test_tensor = torch.tensor(X_test_tensor, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_tensor, dtype=torch.float32)

(390, 50, 200)
(390,)


In [117]:
#print(X_test_tensor[2][129])
#print(y_test_tensor[2][129])

In [151]:




model.eval()

with torch.no_grad():
    predictions = model(X_test_tensor)

# predictions have values between 0 and 1 because forward pass of LSTM contains sigmoid at output
#print(predictions)

predicted_classes = (predictions > 0.5).float() # 0.5 is threshold
#this converts to same dimensional array of True or false, and .float() converts True to 1 and False to 0

#print(predicted_classes)
 

In [152]:
predicted_classes

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

In [119]:
#print(predictions.shape)

In [153]:
# performance metrics
from random import randint

loss = criterion(predictions, y_test_tensor) # use predictions for loss calculation

print(f"Binary Cross-Entropy Loss: {loss.item():.4f}")

def accuracy(y_true, y_pred):
    if y_true.dtype != y_pred.dtype or y_true.shape != y_pred.shape:
        raise ValueError(f"Inputs do not have same type or shape!")
    y_rand = torch.randint(0,2,y_true.shape)
    correct_predictions = (y_true == y_pred).sum().item()
    total_predictions = y_true.numel()
    accuracy = correct_predictions / total_predictions * 100
    return accuracy
accuracy = accuracy(y_test_tensor, predicted_classes)


print(f"Accuracy: {accuracy:.4f}")

#print(y_test_tensor.shape)
#print(predicted_classes.shape)


# Visualization of Actual vs Predicted Classes
# import matplotlib.pyplot as plt
# TODO COULD USE PLT TO VISUALIZE?

Binary Cross-Entropy Loss: 0.6971
Accuracy: 51.0256


# For Kaggle Submission

In [111]:
# RETRAIN MODEL ON ENTIRE TRAINING DATA AND EVALUATE EVAL TWEETS




df_X = pd.concat([df_X_train, df_X_test], ignore_index=True)
df_y = pd.concat([df_y_train, df_y_test], ignore_index=True)




In [112]:
df_X

,MatchID,PeriodID,tweet_vector
0,0,0,"[3098.4608585517853, 6436.185963880271, 363.53..."
1,0,1,"[2493.7816506992094, 5349.19354169257, 284.537..."
2,0,2,"[1972.8186, 4104.4824, 323.55396, -1203.197, -..."
3,0,3,"[2432.1588252037764, 5373.947940472513, 723.78..."
4,0,4,"[2607.9675, 5692.466, 748.0113, -1843.9877, -3..."
...,...,...,...
2132,15,125,"[1768.0021, 3085.3567, 1567.4435, -1976.7213, ..."
2133,15,126,"[1547.5847, 2672.128, 1288.0339, -1692.0323, 4..."
2134,15,127,"[1389.9213, 2417.8992, 1197.792, -1539.6497, 3..."
2135,15,128,"[1233.3035642304458, 2106.0893550515175, 1090...."


In [113]:
df_y

0       0
1       0
2       0
3       0
4       0
       ..
2132    1
2133    1
2134    1
2135    1
2136    0
Name: EventType, Length: 2137, dtype: int64

In [114]:
# convert df_X_test and df_y_test to correct format/dimensions
X_tensor, y_tensor = convert_df_to_3D_tensor(df_X, df_y)
# CONVERT TO PYTORCH TENSOR
X_tensor = torch.tensor(X_tensor, dtype=torch.float32)
y_tensor = torch.tensor(y_tensor, dtype=torch.float32)

(16, 180, 200)
(16, 180)


In [115]:
# NOTES
# HOW TO MAKE SURE THAT we:
# 1. DO NOT ignore the order of the tweets -> (LSTM)
# 2. treat each time period as RELATED to the football match they belong to -> treat each match as a sequence, train LSTM on every sequence
#                      since pytorch tensor expects multiple sequences (batches)



# for LSTM: Each input sequence should consist of tweets from a specific match, ordered by Period ID.
#   tweets of different matches are unrelated, but tweets of a same match are related sequentially (chronologically)
#   structure training data such that tweets are grouped by match id, and ordered by period id